In [4]:
import numpy as np
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import pandas as pd
from plotly.subplots import make_subplots
import pandas as pd
import time
from sklearn.manifold import TSNE, Isomap, LocallyLinearEmbedding, SpectralEmbedding

import plotly_express as px
import plotly.graph_objs as go
import ipywidgets as widgets
import os
from sklearn import preprocessing

In [8]:
min_max_scaler = preprocessing.MinMaxScaler()
df=pd.read_csv("./csvs/f329.csv",error_bad_lines=False)
## drop some stuff
# ['noise','piano','guitar',"voc"]
not_want=['noise','piano','guitar',"voc","5_stack"]
df=df[~df.label.isin(not_want)].reset_index()
df = df[df.columns.drop(list(df.filter(regex='env_shape')))]
df=df.drop(['l'],axis=1)
df["vt"]=pd.Series([[1,2]]*df.shape[0])
##


n_neighbors = 10
X=df.loc[:, ~df.columns.isin(['label','path'])].values

min_max_scaler = preprocessing.MinMaxScaler()
X = np.nan_to_num(X)

# X = min_max_scaler.fit_transform(X)

time_start = time.time()
# tsne = TSNE(n_components=2, random_state=0, perplexity=20, verbose=1)
# tsne = Isomap(n_neighbors, n_components=2)
# tsne = LocallyLinearEmbedding(n_neighbors,n_components=4)
# tsne = SpectralEmbedding(n_neighbors=16,n_components=5,random_state=1)
X_2d = tsne.fit_transform(X)

df2=pd.concat([df["path"],df["label"],pd.Series(X_2d[:,0]),pd.Series(X_2d[:,1])],axis=1)
df2.columns=["path","label","v1","v2"]


AttributeError: 'DataFrame' object has no attribute 'label'

In [45]:
p = px.scatter(df2, x="v1",
                 y="v2",color="label",hover_data=["path"],color_discrete_sequence=px.colors.qualitative.Dark24)
for trace in p.data:
    trace.update(hoverinfo="none",hovertemplate= '')

def hover_fn(trace, points, state):
    if points.point_inds:
        ind = points.point_inds[0]
        drmName=trace.customdata[ind][0][2:]
        filename=os.getcwd()+drmName
        os.system("play %s trim 0 0.5 &" % filename)
        hover_data.value = str(drmName)+"\n"
        
hover_data = widgets.Textarea()  
layout = go.Layout(hovermode=False)
fig  = go.FigureWidget(p)
for f in fig.data:
    f.on_hover(hover_fn)

display(fig,widgets.HBox([hover_data]))

FigureWidget({
    'data': [{'customdata': array([['../dk_data/shake/Household_Shaker_19.wav'],
              …